Graph and Loss visualization using Tensorboard. This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/).




In [1]:
from __future__ import print_function

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

W0825 17:24:47.492006 139680256632704 deprecation.py:323] From <ipython-input-1-d3a4abdef1a1>:6: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0825 17:24:47.493787 139680256632704 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0825 17:24:47.499038 139680256632704 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruction

Extracting /tmp/data/train-images-idx3-ubyte.gz


W0825 17:24:47.805646 139680256632704 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0825 17:24:47.810702 139680256632704 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0825 17:24:47.866682 139680256632704 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: __init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a f

Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
#training hyperparameters
learning_rate = 0.01
training_epochs = 25
batch_size=100
display_step=1
logs_path = '/tmp/tensorflow_logs/example/'

#graph input
x=tf.placeholder(tf.float32, [None, 784], name='InputData')
y=tf.placeholder(tf.float32, [None, 10], name='LabelData')

W=tf.Variable(tf.zeros([784, 10]), name='Weights')
b=tf.Variable(tf.zeros([10]), name='bias')


In [0]:
# construct models encapsulating all ops into scopes,
# making tensorboard's graph visualization more convenient
with tf.name_scope('Model'):
  #model
  pred=tf.nn.softmax(tf.matmul(x,W)+b)
  
with tf.name_scope('Loss'):
  #minimize error using cross entropy
  cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
  
with tf.name_scope('SGD'):
  # gradient descent
  optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
  
with tf.name_scope('Accuracy'):
  #accuracy
  acc = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
  acc = tf.reduce_mean(tf.cast(acc,tf.float32))
  
#initialize variables
init=tf.global_variables_initializer()

#create summary to monitor cost tensor
tf.summary.scalar("loss", cost)

#crate summarey to monitor accuracy tensor
tf.summary.scalar("acc", acc)

#merge all summaries to single op
merged_summary_op=tf.summary.merge_all()

In [4]:
#start training

with tf.Session() as sess:
  sess.run(init)
  
  # op to write logs to tensorboard
  summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
  
  #training cycle
  for epoch in range(training_epochs):
    avg_cost=0.0
    total_batch=int(mnist.train.num_examples/batch_size)
    
    #loop over all batches
    for i in range(total_batch):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      #run optimization op, cost op, and summary nodes
      _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                              feed_dict={x:batch_xs, y:batch_ys})
      #write every log at every iteration
      summary_writer.add_summary(summary, epoch*total_batch+i)
      #compute average loss
      avg_cost += c/total_batch
    
    #display logs per epoch step
    if (epoch+1)%display_step == 0:
      print("epoch",epoch, "cost",avg_cost)
      
  print("optimization finished!")      
  
  #test model
  #calculate accuracy
  print("accuracy",acc.eval({x:mnist.test.images, y:mnist.test.labels}))  

epoch 0 cost 1.1839628335562622
epoch 1 cost 0.6654106751355264
epoch 2 cost 0.5528904629295525
epoch 3 cost 0.4986241849985991
epoch 4 cost 0.46550194491039576
epoch 5 cost 0.4425898587703702
epoch 6 cost 0.4255362641540439
epoch 7 cost 0.4122198114611885
epoch 8 cost 0.40142696800557026
epoch 9 cost 0.3924021376263011
epoch 10 cost 0.3848006728562441
epoch 11 cost 0.3781524218483404
epoch 12 cost 0.37242156638340485
epoch 13 cost 0.36730999133803655
epoch 14 cost 0.3627091372825882
epoch 15 cost 0.3586131681095471
epoch 16 cost 0.3548490467938507
epoch 17 cost 0.3514516370946712
epoch 18 cost 0.3483244306120005
epoch 19 cost 0.34540796637535087
epoch 20 cost 0.34275258985432694
epoch 21 cost 0.34018086089329286
epoch 22 cost 0.33795736968517337
epoch 23 cost 0.3357051495530387
epoch 24 cost 0.3336708156899975
optimization finished!
accuracy 0.9134
